In [100]:
import os
import json
from glob import glob
import pandas as pd
import xml.etree.ElementTree as ET

from tqdm import tqdm
import matplotlib.pyplot as plt

%matplotlib inline

In [25]:
tree = ET.parse('data/stt/arkistosiirto2018/101748201.xml')

In [26]:
root = tree.getroot()

In [58]:
ns = {'n': 'http://iptc.org/std/nar/2006-10-01/'}

In [79]:
# [elem.tag for elem in root.iter()]

In [78]:
# print(ET.tostring(root).decode())

In [77]:
subjects = []
for item in root.findall('n:contentMeta/n:subject/n:name', ns):
    if item.text not in subjects:
        subjects.append(item.text)
subjects

['Kotimaa',
 'Talous',
 'Liikenne',
 'Lentoliikenne',
 'Rahoituspalvelut Liike-elämän palvelut',
 'Kuluttajakäyttäytyminen',
 'Vapaa-aika',
 'Lomamatkailu Turismi',
 'Makrotalous',
 'Kuluttaja-asiat',
 'Yritysinformaatio',
 'Kuluttajat',
 'Matkailu- ja vapaa-aikateollisuus']

In [73]:
body = root.find('n:contentSet/n:inlineXML/n:html/n:body', ns)
for child in body.iter():
    if child.tag == 'body' or child.text is None:
        continue
    print(child.text)





Pitkä viikonloppu Lontoossa loppiaisena 4.1. - 7.1. 2018
Hintoja vertailtiin marraskuun 23. päivänä, jolloin kuvitteellista matkaa suunniteltiin. Menolento aamukoneella ja paluu iltakoneella. Vertailussa oli mukana vain suoria lentoja.


Finnairin lennot: meno klo 8.00 / paluu klo 18.05


Finnairin
- Matkalaukku ruumaan 20 euroa / suunta
- Ikkuna- tai käytäväpaikka 14 euroa / suunta
- Jalkatilapaikka 26 euroa / suunta
Ebookers
- Istuinpaikkapyynnön voi esittää, hinta ei käynyt suoraan ilmi, istuinpaikkakarttaa ei ollut saatavilla
- Matkatavaroiden hinnoittelussa viitattiin Finnairin hinnastoon
Momondon
- Hakupalvelu antoi useiden toimistojen hinnat, joissa haitari oli suuri.
- Jokaisella toimistolla on oma hinnoittelunsa matkatavaroiden, istuinpaikkojen jne. suhteen. Hinnat käyvät ilmi ostotapahtuman edetessä.
- Halvin hinta 249 euroa 
Travel2be.com
- Kallein hinta 361 euroa 
Lol.travel/fi
- Kaikkiaan 15 varaussivustoa myi kyseisen lennon lippua


Norwegianin lennot: meno klo 7.55 

In [75]:
date = root.find('n:contentMeta/n:contentCreated', ns).text

In [94]:
date

'2018-01-01T13:09:17'

In [95]:
title = root.find('n:contentMeta/n:headline', ns).text

In [96]:
title

'EMBARGO Näin hinnat vaihtelevat Helsinki–Lontoo-viikonloppureissulla'

In [121]:
def parse_data(data_dir):
    folders = next(os.walk(data_dir))[1]
    ns = {'n': 'http://iptc.org/std/nar/2006-10-01/'}
    
    for folder in folders:
        articles = []
        files = [x for x in glob(os.path.join(data_dir, folder, '*.xml'))]
        
        for file in tqdm(files):
            tree = ET.parse(file)
            root = tree.getroot()
            title = root.find('n:contentMeta/n:headline', ns).text
            date = root.find('n:contentMeta/n:contentCreated', ns).text
            
            subjects = []
            for item in root.findall('n:contentMeta/n:subject/n:name', ns):
                if item.text not in subjects:
                    subjects.append(item.text)
                    
            
            body = root.find('n:contentSet/n:inlineXML/n:html/n:body', ns)
            text = "" 
            for child in body.iter():
                if child.tag == 'body' or child.text is None:
                    continue
                text += " " + child.text
            
            articles.append({
                'date': date,
                'subjects': subjects,
                'title': title,
                'body': text
            })
            
        with open(os.path.join(data_dir, folder + '.json'), 'w') as out:
            json.dump(articles, out)
    
    return 0

In [120]:
parse_data('data/stt')

100%|██████████| 55529/55529 [00:06<00:00, 8377.83it/s]


0

In [88]:
next(os.walk('data/stt'))[1]

['arkistosiirto2004',
 'arkistosiirto2007',
 'arkistosiirto2014',
 'arkistosiirto2012',
 'arkistosiirto2009',
 'arkistosiirto2000',
 'arkistosiirto2017',
 'arkistosiirto2015',
 'arkistosiirto1999',
 'arkistosiirto2010',
 'arkistosiirto2005',
 'arkistosiirto2016',
 'arkistosiirto1997',
 'arkistosiirto2001',
 'arkistosiirto1994',
 'arkistosiirto1995',
 'arkistosiirto2008',
 'arkistosiirto2002',
 'arkistosiirto1992',
 'arkistosiirto2018',
 'arkistosiirto2013',
 'arkistosiirto2011',
 'arkistosiirto1996',
 'arkistosiirto2003',
 'arkistosiirto2006',
 'arkistosiirto1993',
 'arkistosiirto1998']

In [102]:
#[y for y in glob(os.path.join('data/stt/arkistosiirto2004', '*.xml'))]

In [122]:
with open('data/stt/arkistosiirto1995.json', 'r') as f:
    test = json.load(f)

In [123]:
test[0]

{'date': '1995-06-06T00:00:00',
 'subjects': ['Politiikka'],
 'title': None,
 'body': ' \n Suomi tukee tiukkaa EU-budjettia Välimeri-rahoihin ei vielä kantaa (VN) Helsinki, 7.\u20096. (STT) Suomi tukee EU:n budjetin tiukkaa yleislinjaa ja pitää tarpeellisena leikata menoja kustannustehokkuudeltaan heikoissa kohteissa. Suomen hallitus määritteli keskiviikkona yleiset periaatteensa käsiteltäessä EU:n talousarvioesitystä vuodelle 1996.  Hallituksen kannanotto meni eduskunnan tietoon. Kannanotto määrittää myös valtiovarainministeriön linjaa, kun EU:n neuvosto ryhtyy käsittelemään budjettia heinäkuussa. Suomi ei ole kuitenkaan valmis säästämään kehittämis- ja tutkimustyöstä, Keski- ja Itä-Euroopan tuesta eikä raja-alueiden yhteistyön kehittämisestä. Myös telekommunikaatio, tutkijavaihto ja ympäristöohjelmat kuuluvat Suomen kiinnostuksen kohteisiin EU:n budjetissa. Ensi vuonna alkavien kulttuuriohjelmien rahoitus tulee lisäksi turvata. Maatalousmenoista selvitys ja työllisyyteen lisää Suomi 

In [126]:
def read_data(data_dir, start=1992, end=2018):
    articles = []
    
    for year in range(start, end):
        with open(os.path.join(data_dir, str(year) + '.json')) as f:
            articles += json.load(f)

    return articles

In [127]:
articles = read_data('data/stt_parsed', start=2016, end=2018)

In [128]:
len(articles)

203567

In [130]:
articles[523]

{'date': '2016-08-24T02:12:14',
 'subjects': ['Ulkomaat',
  'Politiikka',
  'Puolustus',
  'Puolustusvoimat',
  'Talous',
  'Metalliteollisuus',
  'Sotateollisuus',
  'Ohjusjärjestelmät'],
 'title': 'Pohjois-Korea testasi jälleen sukellusveneestä laukaistavaa ohjusta',
 'body': ' \n Pohjois-Korea on jälleen testannut ballistisen ohjuksen laukaisua. Se ammuttiin sukellusveneestä maan itärannikolla keskiviikkona. Asiasta kertoo Etelä-Korean uutistoimisto Yonhap. Etelä-Korean puolustushaarakomentajien neuvoston tiedotteessa kerrotaan, että ohjus laukaistiin Japaninmerellä hieman ennen aamukuutta keskiviikkoaamuna paikallista aikaa. Tiedotteessa ei kerrota enempää yksityiskohtia asiasta, eikä myöskään sitä, onnistuiko testi.  Etelä-Korea ja Yhdysvallat aloittivat maanantaina vuotuiset sotaharjoituksensa, mikä on lisännyt jännitteitä maiden ja Pohjois-Korean välillä. Soulista ja Washingtonista on viestitetty harjoitusten olevan puhtaasti puolustuksen harjoittelua, mutta Pohjois-Korea on näh